In [1]:
using Lux, MLUtils, Optimisers, Zygote, OneHotArrays, Random, Statistics, Printf, Reactant
using MLDatasets: MNIST
using SimpleChains: SimpleChains

Reactant.set_default_backend("cpu")

### Construcción de Redes a la Medida

In [2]:
myleakyrelu(x) = leakyrelu(x,0.1)

myleakyrelu (generic function with 1 method)

In [3]:
model = Chain(
    Dense(100,64),
    Dense(64,32),
    Dense(32,60,myleakyrelu)
)

Chain(
    layer_1 = Dense(100 => 64),         # 6_464 parameters
    layer_2 = Dense(64 => 32),          # 2_080 parameters
    layer_3 = Dense(32 => 60, myleakyrelu),  # 1_980 parameters
)         # Total: 10_524 parameters,
          #        plus 0 states.

In [4]:
# Initialize the model parameters and states
rng = Random.default_rng()  # Random number generator
ps, st = Lux.setup(rng, model)

# ps: These are the learnable parameters of your model (like weights and biases in neural networks).
# st: This contains the non-learnable states of your model (like the running mean and variance in BatchNorm).

((layer_1 = (weight = Float32[-0.06478389 -0.11984813 … 0.15681627 0.16947648; -0.025129104 -0.03368413 … 0.01895911 -0.037643936; … ; 0.018977301 -0.14837784 … 0.0028096708 -0.13951942; 0.003441324 -0.013076283 … -0.023756798 -0.10158735], bias = Float32[0.08117366, 0.02158631, 0.063280985, -0.08688009, 0.059794724, -0.060111918, 0.05069244, -0.0048844814, 0.004251182, -0.061550677  …  -0.043969583, 0.0860764, 0.05518954, -0.08183166, -0.024449289, 0.050024725, 0.029975355, -0.097963944, 0.019012714, -0.06459296]), layer_2 = (weight = Float32[0.09718298 0.036273878 … 0.08066933 -0.17278604; -0.19383946 -0.02771119 … 0.015106133 -0.03335275; … ; 0.016515026 0.0021830506 … 0.07616679 -0.18705305; -0.024334371 -0.121246204 … -0.11585476 0.11630356], bias = Float32[0.007469699, 0.06311068, -0.068090394, -0.10777129, 0.049274683, 0.07575385, -0.100262836, -0.09106918, -0.09722954, -0.022584438  …  -0.018853188, 0.11923033, 0.07609905, 0.0009704381, 0.038826898, 0.08527742, -0.02205196, 0.0

In [5]:
# Let's create a dummy input tensor
batch_size = 10  # Example batch size
dummy_input = rand(Float32, 100, batch_size)  # A random matrix of size (100, batch_size)

# Pass the dummy input through the model
output = model(dummy_input, ps, st)

output[1]# Let's create a dummy input tensor
batch_size = 10  # Example batch size
dummy_input = rand(Float32, 100, batch_size)  # A random matrix of size (100, batch_size)

# Pass the dummy input through the model
output = model(dummy_input, ps, st)

output[1]

60×10 Matrix{Float32}:
  0.769751      0.727672    …   0.341455     0.868212    0.416415
 -0.0544012    -0.0629462      -0.0641284   -0.0492751  -0.0192534
 -0.00119533   -0.0843699      -0.0377123   -0.0624201  -0.0430898
 -0.0238029     0.236865        0.699302     0.436931    0.0404484
  0.487747      0.602919        0.272539     0.310568    0.14406
  0.962866      0.859761    …  -0.0130674    0.467875   -0.0294352
  0.364825      0.380429       -0.00248813   0.207294   -0.0203249
  0.405412      0.251979        0.225704     0.333966   -0.0252425
 -0.0219704    -0.00878765      0.00729539  -0.040548    1.15106
 -0.103623     -0.121596       -0.0638875   -0.118568   -0.0815456
  ⋮                         ⋱                           
  0.19238      -0.00344709     -0.0402973   -0.0231743  -0.0674198
  0.502578      1.02678         1.40052      0.840528    0.972065
 -0.00706753   -0.0540543      -0.0400687   -0.0557558  -0.0388777
  0.0800557     0.699013       -0.00579532   0.356491  

In [6]:
# Example data: an array of tuples [(x, y), ...]
data = [(rand(Float32, 100), rand(Float32, 10)) for _ in 1:128]  # Example with 128 samples

# Extract inputs (x) and outputs (y) from the tuples
x_batch = cat(map(t -> t[1], data)...; dims=2)  # Stack inputs along the 4th dimension (batch)
y_batch = cat(map(t -> t[2], data)...; dims=2)  # Stack outputs along the 2nd dimension (batch)

# Check the shapes of the resulting batches
println("Shape of x_batch: ", size(x_batch))  # Should be (28, 28, 1, batch_size)
println("Shape of y_batch: ", size(y_batch))  # Should be (10, batch_size)

Shape of x_batch: (100, 128)
Shape of y_batch: (10, 128)


In [30]:
typeof(data)

Vector{Tuple{Vector{Float32}, Vector{Float32}}} (alias for Array{Tuple{Array{Float32, 1}, Array{Float32, 1}}, 1})